# Project: Chatbot with Audio
#### This project creates a chat only via audio. The user uses microphone to ask question and the chatbot answer with text and audio.

In [1]:
import gradio as gr
from openai import OpenAI
import pyttsx3
import os
import json
from dotenv import load_dotenv
import whisper
# Initialization
from pydub import AudioSegment
import os
import subprocess
from io import BytesIO
import tempfile
from pydub.playback import play
from gtts import gTTS


load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [2]:
whisper_model = whisper.load_model("base")
def transcribe_audio(audio_path):    
    # print("Transcribing audio...")
    result = whisper_model.transcribe(audio_path, language="en")
    return result["text"]


def talker(message):
    # Mocked OpenAI response for testing
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )
    
    # Handle audio stream
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    
    # Play the audio
    # play_audio_with_ffplay(audio, custom_temp_dir)
    play(audio)
    

/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locati

In [ ]:

# Global variable to store chat history
conversation = []
chat_history = []

# Function to transcribe audio and update chat history
def update_chat(audio_path):
    global conversation, chat_history
    # import pdb; pdb.set_trace()
    # Step 1: Transcribe audio
    with open(audio_path, "rb") as audio_file:
        transcription =openai.audio.transcriptions.create(model="whisper-1", file = audio_file)
    user_input = transcription.text#["text"]

    # Add user input to conversation and chat history
    conversation.append({"role": "user", "content": user_input})
    chat_history.append(("User", user_input))

    # Step 2: Generate response using ChatGPT
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation
    )
    assistant_response = response.choices[0].message.content

    # Add assistant response to conversation and chat history
    conversation.append({"role": "assistant", "content": assistant_response})
    chat_history.append(("Bot", assistant_response))

    # Return updated chat history (audio will be added later)
    return chat_history, None  # No audio at this step

# Function to generate audio for the latest bot response
def generate_audio():
    global chat_history
    # import pdb; pdb.set_trace()

    # Get the last bot response
    if chat_history:
        last_bot_response = chat_history[-1][1]
    else:
        last_bot_response = "No response available."

    # Generate TTS for the bot response
    tts = gTTS(last_bot_response)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)
    talker(last_bot_response)

    # Return the audio file path
    return temp_file.name

# Gradio interface setup
with gr.Blocks() as interface:
    chatbot = gr.Chatbot(label="Chat History")
    audio_input = gr.Audio(sources="microphone", type="filepath", label="Speak")
    audio_output = gr.Audio(label="Bot Response")
    submit_button = gr.Button("Submit")

    # Define the workflow
    submit_button.click(
        update_chat,
        inputs=[audio_input],
        outputs=[chatbot, audio_output],  # Update chat history first
    ).then(
        generate_audio,
        inputs=[],
        outputs=[audio_output],  # Play the audio afterward
    )

# Launch the Gradio app
interface.launch(debug=True)


/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/tmp/tmpwq5uwta3.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:01.99, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.93 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B f=0/0   

Traceback (most recent call last):
  File "/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/httpx/_transports/default.py", line 72, in map_httpcore_exceptions
    yield
  File "/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/httpx/_transports/default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 256, in handle_request
    raise exc from None
  File "/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    raise exc
  File "/home/morteza/miniconda3/envs/llms/lib/python3.11/site-packages/http